---

# Est-ce encore loin Grand Schtroumpf?
### GoPiGo 101 - Série d'exercices 5
##### Manipulation du **télémètre** (capteur de distance).

Pour cet exercice, vous devez avoir installé et connecté le télémètre sur l'un des ports `I2C`. [Voir cette image](https://gopigo3.readthedocs.io/en/master/api-basic/structure.html#hardware-ports).

---

Les fonctions suivantes permettent la manipulation du capteur de distance :
 - `EasyGoPiGo3.init_distance_sensor` retourne un objet `EasyDistanceSensor`
 - `EasyDistanceSensor.read_mm` retourne la distance lue en millimètre
     - domaine en millimètre : [5, 2300]
     - valeur obtenue sans cible détectée : 3000
     - précision : ~1 millimètre
 - `EasyDistanceSensor.read` retourne la distance lue en centimètre
     - domaine en centimètre : [0, 230]
     - valeur obtenue sans cible détectée : 300
     - précision : ~1 centimètre
 - `EasyDistanceSensor.read_inches` retourne la distance lue en pouces
     - domaine en pouce : [0, 90]
     - valeur obtenue sans cible détectée : > 90
     - précision : ~0.1"
 
**Important** : Vous remarquerez que la performance associée à l'usage du télémètre est intéressante mais limitée. D'ailleurs, on peut observer :
 - le temps de réponse est relativement lent
 - la résolution n'est peut-être pas aussi fine qu'on le voudrait mais au moins elle est connue

### Démonstration

In [ ]:
import easygopigo3 as gpg
import time

robot = gpg.EasyGoPiGo3()
distance_sensor_port = 'I2C'
distance_sensor = robot.init_distance_sensor(distance_sensor_port)
separator_length = 80

print(f'La distance en centimètre est {distance_sensor.read()} cm')
print(f' - retourne un nombre entier entre 0 et 230 centimètres')
print(f' - la valeur 300 correspond à une lecture hors de portée')
print('-' * separator_length)
print(f'La distance en millimètre est {distance_sensor.read_mm()} mm')
print(f' - retourne un nombre entier entre 5 et 2300 centimètres')
print(f' - la valeur 3000 correspond à une lecture hors de portée')
print('-' * separator_length)
print(f'La distance en pouces est : {distance_sensor.read_inches()}"')
print(f' - retourne un nombre réel entre 0. et 90. pouces (avec une précision de 1 chiffre)')
print(f' - une valeur supérieure à 90 correspond à une lecture hors de portée')

del distance_sensor
del distance_sensor_port
del robot
del time
del gpg

---
### Préparation
Faites la mise en place du code commun pour cette série d'exercices

In [ ]:
# Mise en place du code commun
import easygopigo3 as gpg
import time
import math

robot = gpg.EasyGoPiGo3()
remote_control_port = 'AD1'
remote_control = robot.init_remote(port=remote_control_port)
distance_sensor_port = 'I2C'
distance_sensor = robot.init_distance_sensor(distance_sensor_port)

---
### Exercice 5.1.
Faites un programme qui affiche la distance mesurée en millimètre le plus rapidement possible pendant _x_ secondes.

In [ ]:
# Solution
print('Mesure de la distance en cours...')

duration = 10.0
ref_counter = time.perf_counter()
out_of_range_message = 'cible hors de portée'

while time.perf_counter() <= ref_counter + duration:
    distance = distance_sensor.read_mm()
    text = f'{distance:>5} mm' if distance != 3000 else out_of_range_message
    print(f'\r{text}', end=' ' * 20)
        
print('\n... fin de la mesure de distance!')

---
### Exercice 5.2.
Ajouter à l'exercice précédent les options suivantes avec la télécommande :
 - `1` : lecture en kilomètre
 - `2` : lecture en centimètre (valeur par défaut)
 - `3` : lecture en millimètre
 - `4` : lecture en pouce (décimale numérique, exemple : 3.25 pouce)
 - `5` : lecture en pied (décimale numérique, exemple : 0.125 pied)
 - `ok` : fin du programme (il n'y a plus de délais)
 
_En option_ (* §) pour les étudiants intéressés par le sujet : 
 - `6` : une lecture en pied, pouce et fraction la plus précise jusqu'à 1/32 (exemple : 1' 3" 3/16).

In [ ]:
# Solution
print('Mesure de la distance en cours...')

key = ''
distance_mode = 2
out_of_range_message = 'cible hors de portée'

while key != 'ok':
    key = remote_control.get_remote_code()
    if key == 'ok':
        continue
        
    if key >= '1' and key <= '6':
        distance_mode = int(key)
        
    if distance_mode == 1:
        distance = distance_sensor.read()
        text = f'{distance / 100.0 / 1000.0:0.6f} km' if distance != 300 else out_of_range_message
    elif distance_mode == 2:
        distance = distance_sensor.read()
        text = f'{distance} cm' if distance != 300 else out_of_range_message
    elif distance_mode == 3:
        distance = distance_sensor.read_mm()
        text = f'{distance} mm' if distance != 3000 else out_of_range_message
    elif distance_mode == 4:
        distance = distance_sensor.read_inches()
        text = f'{distance}"' if distance <= 90 else out_of_range_message
    elif distance_mode == 5:
        distance = distance_sensor.read_inches()
        text = f'{distance / 12.:0.3f}\'' if distance <= 90 else out_of_range_message
    elif distance_mode == 6:
        text = f'À faire en option...'

    print(f'\r{text}', end=' ' * 40)
    
print('\n... fin de la mesure de distance!')

---
### Exercice 5.3.
Ajoutez à l'exercice précédent l'affichage de la fréquence de la boucle du programme : les _FPS_.

In [ ]:
# Solution

from di_sensors.easy_distance_sensor import EasyDistanceSensor

print('Mesure de la distance en cours...')

key = ''
distance_mode = 2

fct_to_read = [EasyDistanceSensor.read, EasyDistanceSensor.read, EasyDistanceSensor.read_mm, EasyDistanceSensor.read_inches, EasyDistanceSensor.read_inches]
scale_factor = [1. / 100. / 1000., 1., 1., 1., 12.]
units = [' km', ' cm', ' mm', '"', "'"]
format = ['', '', '', '', '0.3f']
out_of_range = { EasyDistanceSensor.read:300, EasyDistanceSensor.read_mm:3000, EasyDistanceSensor.read_inches:91 }
out_of_range_message = 'cible hors de portée'

previous_counter = time.perf_counter()

while key != 'ok':
    now = time.perf_counter()
    elapsed_time = now - previous_counter
    fps = 1. / elapsed_time if elapsed_time else -1.
    previous_counter = now
    
    key = remote_control.get_remote_code()
    if key == 'ok':
        continue
        
    if key >= '1' and key <= '5': # 6 option
        distance_mode = int(key) - 1
    
    distance = fct_to_read[distance_mode](distance_sensor)
    if distance < out_of_range[fct_to_read[distance_mode]]:
        distance *= scale_factor[distance_mode]
        text = f'{distance:{format[distance_mode]}}{units[distance_mode]}'
    else:
        text = out_of_range_message
    print(f'\r{fps:0.1f} fps : {text}', end=' ' * 40)
    
print('\n... fin de la mesure de distance!')

---
### Exercice 5.4.
Faites l'acquisition de 100 valeurs de distance en millimètre tout en prenant le soins de déplacer manuellement une cible devant le télémètre. 

Ensuite, faites afficher les statistiques suivantes :
 - le nombre total de mesures effectuées
 - le pourcentage de bonnes mesures et de mesures hors de portée
 - seulement pour les lectures qui ne sont pas hors de portée :
     - le nombre de mesures
     - la valeur minimum
     - la valeur maximum
     - la valeur moyenne
     - l'écart type
     - la valeur médianne
     - l'étendu

In [ ]:
# Solution

# Solution simpliste sans numpy -> une solution avec numpy serait plus élégante et serait à faire en exercice supplémentaire
print('Mesures de distance en cours...')

progress_char_length = 50
measures = []
number_of_measures = 100
out_of_range_distance = 3000
minimum = 3000
maximum = 0
sum = 0

for i in range(number_of_measures):
    distance = distance_sensor.read_mm()
    if distance < out_of_range_distance:
        measures.append(distance)
        
        if distance < minimum:
            minimum = distance
        if distance > maximum:
            maximum = distance
        sum += distance

    progress_head = round(i / 100 * progress_char_length)
    progress_tail = progress_char_length - progress_head
    distance_head = round(distance / maximum * progress_char_length)
    distance_tail = progress_char_length - distance_head
    print(f'\r| {i+1:>03}% {"+" * progress_head}{"-" * progress_tail} |   | {distance:>04} mm {"+" * distance_head}{"-" * distance_tail} |', end=' ' * 110)
    
if measures:
    average = sum / len(measures)
    stdev = 0
    for value in measures:
        stdev += (value - average) ** 2
    stdev = math.sqrt(stdev / (len(measures) - 1))
    
    measures.sort()
    if len(measures) % 2 == 1:
        median = measures[len(measures)//2]
    else:
        median = (measures[len(measures)//2] + measures[len(measures)//2 + 1]) / 2
else:
    minimum = 0
    maximum = 0
    sum = 0    
    average = 0
    stdev = 0
    median = 0

print(f'''
Le nombre de mesures effectués : {number_of_measures}
  > {len(measures) / number_of_measures * 100:5.1f} % de bonnes mesures
  > {(number_of_measures - len(measures)) / number_of_measures * 100:5.1f} % de mesures hors bornes

Statistiques
 - nombre de mesures : {len(measures):>4}
 - minimum           : {minimum:8.3f} mm
 - maximum           : {maximum:8.3f} mm
 - moyenne           : {average:8.3f} mm
 - écart type        : {stdev:8.3f} mm
 - médiane           : {median:8.3f} mm
 - étendu            : {maximum - minimum:8.3f} mm''')

print('\n... fin de la mesure de distance!')